In [31]:
import os
import json
import math
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple, Optional
from darts import TimeSeries
from darts.models import RNNModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mae, mape, rmse
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def build_monthly_features(bank_df: pd.DataFrame) -> pd.DataFrame:
    """Build monthly cashflow and balance features from bank transactions."""
    
    df = bank_df.copy()
    
    # Handle date column - could be index or column
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
    elif df.index.name == 'date' or isinstance(df.index, pd.DatetimeIndex):
        df = df.reset_index()
        if 'date' not in df.columns:
            df.rename(columns={df.columns[0]: 'date'}, inplace=True)
        df['date'] = pd.to_datetime(df['date'])
    else:
        raise ValueError("No date column or datetime index found in dataframe")
    
    # Ensure required columns exist
    required_cols = ['income', 'expense']
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f"Bank dataframe must contain columns: {required_cols}")
    
    # Convert to numeric, handling any non-numeric values
    df['income'] = pd.to_numeric(df['income'], errors='coerce').fillna(0)
    df['expense'] = pd.to_numeric(df['expense'], errors='coerce').fillna(0)
    
    # Create monthly aggregations
    df['month'] = df['date'].dt.to_period('M')
    
    monthly_agg = df.groupby('month').agg({
        'income': 'sum',
        'expense': 'sum'
    }).reset_index()
    
    # Calculate derived features
    monthly_agg['net_cashflow'] = monthly_agg['income'] - monthly_agg['expense']
    monthly_agg['balance'] = monthly_agg['net_cashflow'].cumsum()
    
    # Convert period to timestamp for compatibility with Darts
    monthly_agg['date'] = monthly_agg['month'].dt.to_timestamp('M')  # End of month
    monthly_agg = monthly_agg.set_index('date')
    
    # Fill missing months with zeros if needed
    if len(monthly_agg) > 1:
        full_idx = pd.date_range(
            start=monthly_agg.index.min(), 
            end=monthly_agg.index.max(), 
            freq='ME'  # Month end
        )
        monthly_agg = monthly_agg.reindex(full_idx, fill_value=0.0)
        # Recalculate balance after filling gaps
        monthly_agg['balance'] = monthly_agg['net_cashflow'].cumsum()
    
    return monthly_agg[['net_cashflow', 'balance']]

def load_user_data(base_dir: str, user_id: str) -> pd.DataFrame:
    """Load and combine bank and card data for a user."""
    
    bank_file = os.path.join(base_dir, f"bank_user_{user_id}.csv")
    card_file = os.path.join(base_dir, f"card_user_{user_id}.csv")
    
    frames = []
    
    # Load bank data
    if os.path.exists(bank_file):
        bank_df = pd.read_csv(bank_file)
        frames.append(bank_df)
    
    # Load card data if available
    if os.path.exists(card_file):
        card_df = pd.read_csv(card_file)
        frames.append(card_df)
    
    if not frames:
        raise FileNotFoundError(f"No data files found for user {user_id}")
    
    # Combine all data
    combined_df = pd.concat(frames, ignore_index=True)
    return combined_df


def generate_synthetic_data(real_data: pd.DataFrame, n_months: int = 24, noise_level: float = 0.1) -> pd.DataFrame:
    """Generate synthetic data by extending patterns from real data."""
    
    if len(real_data) < 2:
        raise ValueError("Need at least 2 months of real data to generate synthetic data")
    
    # Calculate growth trends and seasonality
    cashflow_trend = real_data['net_cashflow'].diff().mean()
    balance_trend = real_data['balance'].diff().mean()
    
    # Base statistics for noise generation
    cf_std = real_data['net_cashflow'].std()
    bal_std = real_data['balance'].std()
    
    # Generate future dates
    start_date = real_data.index[-1] + pd.DateOffset(months=1)
    future_dates = pd.date_range(start=start_date, periods=n_months, freq='ME')
    
    # Initialize synthetic data
    synthetic_data = []
    last_balance = real_data['balance'].iloc[-1]
    
    np.random.seed(42)  # For reproducibility
    
    for i, date in enumerate(future_dates):
        # Add trend and seasonal patterns
        seasonal_factor = 1 + 0.1 * np.sin(2 * np.pi * i / 12)  # Annual seasonality
        
        # Generate cashflow with trend and noise
        base_cf = real_data['net_cashflow'].mean() + (cashflow_trend * i)
        noise_cf = np.random.normal(0, noise_level * cf_std)
        net_cashflow = (base_cf * seasonal_factor) + noise_cf
        
        # Update balance
        last_balance += net_cashflow
        noise_bal = np.random.normal(0, noise_level * bal_std * 0.1)  # Smaller noise for balance
        balance = last_balance + noise_bal
        
        synthetic_data.append({
            'net_cashflow': net_cashflow,
            'balance': balance
        })
    
    synthetic_df = pd.DataFrame(synthetic_data, index=future_dates)
    return synthetic_df


def build_datasets(data_dir: str, holdout_frac: float = 0.1, min_months: int = 3, seed: int = 42):
    """Build training and testing datasets."""
    
    # Generate user IDs
    user_ids = [f"{i:04d}" for i in range(1, 1001)]
    
    # Split users into seen/unseen
    np.random.seed(seed)
    n_unseen = max(1, int(len(user_ids) * holdout_frac))
    unseen_users = set(np.random.choice(user_ids, size=n_unseen, replace=False))
    seen_users = [uid for uid in user_ids if uid not in unseen_users]
    
    datasets = {
        'seen_users': seen_users,
        'unseen_users': sorted(list(unseen_users)),
        'real_series': {'seen': [], 'unseen': []},
        'synthetic_series': {'seen': []},
        'train_series': [],
        'test_series': {'seen': [], 'unseen': []}
    }
    
    valid_users = {'seen': [], 'unseen': []}
    
    for user_id in user_ids:
        try:
            # Load user data
            user_data = load_user_data(data_dir, user_id)
            monthly_features = build_monthly_features(user_data)
            
            # Skip users with insufficient data
            if len(monthly_features) < min_months:
                continue
                
            # Limit to first 3 months for consistency
            monthly_features = monthly_features.head(3)
            # Create TimeSeries for both features
            ts_data = TimeSeries.from_dataframe(
                monthly_features.reset_index(), 
                time_col='date',
                value_cols=['net_cashflow', 'balance']
            )
            
            user_type = 'seen' if user_id in seen_users else 'unseen'
            datasets['real_series'][user_type].append(ts_data)
            valid_users[user_type].append(user_id)
            
            # Generate synthetic data for seen users
            if user_type == 'seen':
                synthetic_monthly = generate_synthetic_data(monthly_features, n_months=24)
                # Combine real and synthetic
                extended_data = pd.concat([monthly_features, synthetic_monthly])
                ts_synthetic = TimeSeries.from_dataframe(
                    extended_data.reset_index(),
                    time_col='date', 
                    value_cols=['net_cashflow', 'balance']
                )
                datasets['synthetic_series']['seen'].append(ts_synthetic)
                datasets['train_series'].append(ts_synthetic)
            
            # Prepare test series (first 2 months for prediction)
            if len(ts_data) >= 2:
                datasets['test_series'][user_type].append(ts_data[:2])
                
        except Exception as e:
            print(f"Warning: Skipping user {user_id} due to error: {e}")
            continue
    
    # Update user lists to only include valid users
    datasets['seen_users'] = valid_users['seen']
    datasets['unseen_users'] = valid_users['unseen']
    
    print(f"Valid users: seen={len(datasets['seen_users'])}, unseen={len(datasets['unseen_users'])}")
    print(f"Training series: {len(datasets['train_series'])}")
    
    return datasets


def train_forecasting_model(train_series: List[TimeSeries], 
                          input_chunk_length: int = 12,
                          output_chunk_length: int = 6,
                          epochs: int = 100,
                          seed: int = 42) -> RNNModel:
    """Train a multi-variate RNN model for cashflow and balance forecasting."""
    
    model = RNNModel(
        model="LSTM",
        hidden_dim=128,
        n_rnn_layers=3,
        dropout=0.2,
        batch_size=32,
        n_epochs=epochs,
        optimizer_kwargs={"lr": 1e-3},
        model_name="CashflowBalanceForecaster",
        random_state=seed,
        input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        force_reset=True,
        save_checkpoints=True
    )
    
    print(f"Training model on {len(train_series)} series...")
    model.fit(train_series, verbose=True)
    
    return model


def evaluate_forecasts(model: RNNModel, 
                      test_series: List[TimeSeries], 
                      actual_series: List[TimeSeries],
                      horizons: List[int] = [3, 6, 12, 24]) -> Dict:
    """Evaluate forecasting performance across multiple horizons."""
    
    results = {}
    
    for horizon in horizons:
        predictions = []
        actuals = []
        
        for test_ts, actual_ts in zip(test_series, actual_series):
            try:
                # Make prediction
                pred = model.predict(n=horizon, series=test_ts, num_samples=1)
                
                # Get actual values (limited to available data)
                actual_len = min(horizon, len(actual_ts) - len(test_ts))
                if actual_len > 0:
                    actual_subset = actual_ts[len(test_ts):len(test_ts) + actual_len]
                    pred_subset = pred[:actual_len]
                    
                    predictions.append(pred_subset)
                    actuals.append(actual_subset)
                    
            except Exception as e:
                print(f"Warning: Prediction failed for horizon {horizon}: {e}")
                continue
        
        if predictions and actuals:
            # Calculate metrics for each component
            metrics = {}
            for i, component in enumerate(['net_cashflow', 'balance']):
                try:
                    pred_component = [p.univariate_component(i) for p in predictions]
                    actual_component = [a.univariate_component(i) for a in actuals]
                    
                    metrics[f'{component}_MAE'] = np.mean([
                        mae(actual, pred) for actual, pred in zip(actual_component, pred_component)
                    ])
                    metrics[f'{component}_MAPE'] = np.mean([
                        mape(actual, pred) for actual, pred in zip(actual_component, pred_component)
                    ])
                    metrics[f'{component}_RMSE'] = np.mean([
                        rmse(actual, pred) for actual, pred in zip(actual_component, pred_component)
                    ])
                except Exception as e:
                    print(f"Warning: Metric calculation failed for {component}: {e}")
                    metrics[f'{component}_MAE'] = float('nan')
                    metrics[f'{component}_MAPE'] = float('nan')
                    metrics[f'{component}_RMSE'] = float('nan')
            
            results[f'horizon_{horizon}'] = metrics
        else:
            results[f'horizon_{horizon}'] = {
                'net_cashflow_MAE': float('nan'),
                'net_cashflow_MAPE': float('nan'),
                'net_cashflow_RMSE': float('nan'),
                'balance_MAE': float('nan'),
                'balance_MAPE': float('nan'),
                'balance_RMSE': float('nan')
            }
    
    return results


def save_forecasts(model: RNNModel, 
                   test_series: List[TimeSeries], 
                   user_ids: List[str],
                   horizons: List[int],
                   output_dir: str):
    """Generate and save forecasts for all horizons."""
    
    os.makedirs(output_dir, exist_ok=True)
    
    for horizon in horizons:
        all_forecasts = []
        
        for i, (test_ts, user_id) in enumerate(zip(test_series, user_ids)):
            try:
                forecast = model.predict(n=horizon, series=test_ts, num_samples=1)
                
                # Convert to dataframe
                forecast_df = forecast.pd_dataframe().reset_index()
                forecast_df['user_id'] = user_id
                forecast_df['horizon'] = horizon
                
                all_forecasts.append(forecast_df)
                
            except Exception as e:
                print(f"Warning: Failed to generate forecast for user {user_id}, horizon {horizon}: {e}")
                continue
        
        if all_forecasts:
            combined_forecasts = pd.concat(all_forecasts, ignore_index=True)
            output_file = os.path.join(output_dir, f'forecasts_horizon_{horizon}.csv')
            combined_forecasts.to_csv(output_file, index=False)
            print(f"Saved forecasts for horizon {horizon} to {output_file}")



In [36]:
def main():
    """Main pipeline execution."""
    
    # Configuration
    DATA_DIR = "./data/synthetic_users/"
    OUTPUT_DIR = "./results/outputs/"
    HORIZONS = [3, 6, 12, 24]
    
    print("=== Cashflow & Balance Forecasting Pipeline ===")
    
    # Build datasets
    print("\n1. Building datasets...")
    datasets = build_datasets(DATA_DIR, holdout_frac=0.1, min_months=3, seed=42)
    
    if not datasets['train_series']:
        raise RuntimeError("No valid training data found")
    
    # Scale the data
    print("\n2. Scaling data...")
    scaler = Scaler()
    
    # Fit scaler on real seen data only
    real_seen = datasets['real_series']['seen']
    if real_seen:
        scaler.fit(real_seen)
        
        # Transform all series
        train_series_scaled = scaler.transform(datasets['train_series'])
        test_seen_scaled = scaler.transform(datasets['test_series']['seen'])
        test_unseen_scaled = scaler.transform(datasets['test_series']['unseen'])
        real_seen_scaled = scaler.transform(real_seen)
        real_unseen_scaled = scaler.transform(datasets['real_series']['unseen'])
    else:
        raise RuntimeError("No real seen data available for scaling")
    
    # Train model
    print("\n3. Training forecasting model...")
    model = train_forecasting_model(
        train_series_scaled,
        input_chunk_length=6,  # Use 6 months of history
        output_chunk_length=3,  # Predict 3 months ahead
        epochs=50,
        seed=42
    )
    
    # Evaluate on seen users
    print("\n4. Evaluating on seen users...")
    metrics_seen = evaluate_forecasts(
        model, test_seen_scaled, real_seen_scaled, horizons=HORIZONS
    )
    
    # Evaluate on unseen users
    print("\n5. Evaluating on unseen users...")
    metrics_unseen = evaluate_forecasts(
        model, test_unseen_scaled, real_unseen_scaled, horizons=HORIZONS
    )
    
    # Save results
    print("\n6. Saving results...")
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    results = {
        'dataset_info': {
            'n_seen_users': len(datasets['seen_users']),
            'n_unseen_users': len(datasets['unseen_users']),
            'n_training_series': len(datasets['train_series'])
        },
        'metrics': {
            'seen_users': metrics_seen,
            'unseen_users': metrics_unseen
        },
        'horizons': HORIZONS
    }
    
    with open(os.path.join(OUTPUT_DIR, 'evaluation_results.json'), 'w') as f:
        json.dump(results, f, indent=2)
    
    # Generate and save forecasts
    print("\n7. Generating forecasts...")
    save_forecasts(
        model, 
        test_seen_scaled,
        datasets['seen_users'][:len(test_seen_scaled)], 
        HORIZONS, 
        OUTPUT_DIR
    )
    
    # Save user lists
    with open(os.path.join(OUTPUT_DIR, 'seen_users.json'), 'w') as f:
        json.dump(datasets['seen_users'], f, indent=2)
    
    with open(os.path.join(OUTPUT_DIR, 'unseen_users.json'), 'w') as f:
        json.dump(datasets['unseen_users'], f, indent=2)
    
    print(f"\n=== Pipeline Complete ===")
    print(f"Results saved to: {OUTPUT_DIR}")
    print(f"Seen users performance: {len(datasets['seen_users'])} users")
    print(f"Unseen users performance: {len(datasets['unseen_users'])} users")
    
    # Print summary metrics
    print(f"\nSample Metrics (Horizon 3):")
    if 'horizon_3' in metrics_seen:
        seen_h3 = metrics_seen['horizon_3']
        print(f"Seen Users - Cashflow MAE: {seen_h3.get('net_cashflow_MAE', 'N/A'):.2f}")
        print(f"Seen Users - Balance MAE: {seen_h3.get('balance_MAE', 'N/A'):.2f}")
    
    if 'horizon_3' in metrics_unseen:
        unseen_h3 = metrics_unseen['horizon_3']
        print(f"Unseen Users - Cashflow MAE: {unseen_h3.get('net_cashflow_MAE', 'N/A'):.2f}")
        print(f"Unseen Users - Balance MAE: {unseen_h3.get('balance_MAE', 'N/A'):.2f}")


In [41]:
if __name__ == '__main__':
    main()

AttributeError: time_col='date' is not present.


AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.


=== Cashflow & Balance Forecasting Pipeline ===

1. Building datasets...
            net_cashflow  balance
2023-01-31       -743.54  -743.54
2023-02-28       -648.49 -1392.03
2023-03-31       -303.61 -1695.64
            net_cashflow  balance
2023-01-31       3965.11  3965.11
2023-02-28       1179.58  5144.69
2023-03-31       3042.82  8187.51
            net_cashflow  balance
2023-01-31       2172.07  2172.07
2023-02-28       1872.67  4044.74
2023-03-31       1967.48  6012.22
            net_cashflow  balance
2023-01-31        509.13   509.13
2023-02-28        604.27  1113.40
2023-03-31        512.39  1625.79
            net_cashflow   balance
2023-01-31       3634.27   3634.27
2023-02-28       3843.01   7477.28
2023-03-31       4461.47  11938.75
            net_cashflow   balance
2023-01-31       5044.76   5044.76
2023-02-28       3121.03   8165.79
2023-03-31       4267.80  12433.59
            net_cashflow  balance
2023-01-31       2893.23  2893.23
2023-02-28       3496.24  6389.47
2

AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.


            net_cashflow   balance
2023-01-31       3807.83   3807.83
2023-02-28       4228.01   8035.84
2023-03-31       3160.24  11196.08
            net_cashflow  balance
2023-01-31        958.23   958.23
2023-02-28        658.69  1616.92
2023-03-31        149.29  1766.21
            net_cashflow  balance
2023-01-31       1600.26  1600.26
2023-02-28        110.29  1710.55
2023-03-31        128.69  1839.24
            net_cashflow  balance
2023-01-31        263.55   263.55
2023-02-28        309.56   573.11
2023-03-31        998.38  1571.49
            net_cashflow  balance
2023-01-31       2794.06  2794.06
2023-02-28       2151.17  4945.23
2023-03-31       2223.44  7168.67
            net_cashflow  balance
2023-01-31       1217.21  1217.21
2023-02-28       2974.48  4191.69
2023-03-31       2321.27  6512.96
            net_cashflow  balance
2023-01-31       2033.40  2033.40
2023-02-28       1726.83  3760.23
2023-03-31      -1069.88  2690.35
            net_cashflow   balance
2023-01-3

...

AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.


            net_cashflow  balance
2023-01-31       2816.52  2816.52
2023-02-28       3929.98  6746.50
2023-03-31       2484.84  9231.34
            net_cashflow  balance
2023-01-31       1080.37  1080.37
2023-02-28       1412.66  2493.03
2023-03-31        -82.49  2410.54
            net_cashflow  balance
2023-01-31       2270.84  2270.84
2023-02-28       1729.37  4000.21
2023-03-31       2696.68  6696.89
            net_cashflow  balance
2023-01-31        392.61   392.61
2023-02-28       -133.74   258.87
2023-03-31       -373.26  -114.39
            net_cashflow   balance
2023-01-31       3767.48   3767.48
2023-02-28       4629.26   8396.74
2023-03-31       5111.36  13508.10
            net_cashflow  balance
2023-01-31       1049.26  1049.26
2023-02-28       2664.68  3713.94
2023-03-31        989.28  4703.22
            net_cashflow  balance
2023-01-31        401.19   401.19
2023-02-28        809.91  1211.10
2023-03-31        309.20  1520.30
            net_cashflow  balance
2023-01-31

AttributeError: time_col='date' is not present.
AttributeError: time_col='date' is not present.


            net_cashflow  balance
2023-01-31       2119.22  2119.22
2023-02-28       1263.02  3382.24
2023-03-31       2182.63  5564.87
            net_cashflow  balance
2023-01-31       2557.75  2557.75
2023-02-28       2759.98  5317.73
2023-03-31       1989.38  7307.11


KeyboardInterrupt: 